In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR

In [ ]:
data = pd.read_excel(f'data(SA){20240429}.xlsx', index_col=0)
data.index = pd.PeriodIndex(data.index, freq='M')

In [ ]:
endog = data[['GDP(SA)', '전산업생산지수(SA)']]
endog = np.log(endog)
endog = endog.groupby(endog.index.asfreq('Q')).mean()
endog = endog.diff()*100

In [ ]:
endog['회사채3년AA국채3년금리차'] = data['회사채3년AA국채3년금리차'].groupby(data.index.asfreq('Q')).mean()

In [ ]:
endog = endog.dropna()

In [ ]:
results = pd.DataFrame(
    index=pd.period_range(start='2008Q1', end='2015Q4', freq='Q'),
    columns=[
        'var_nowcasting',
        'var_nowcasting_1Q_ahead'
        ]
    )

In [ ]:
for period in results.index:
    realtimeEndog = endog.loc[:period - 1].copy()
    mdl = VAR(realtimeEndog).fit()
    fcst = mdl.forecast(y=realtimeEndog.iloc[-mdl.k_ar:].to_numpy(), steps=1)
    results.loc[period, 'var_nowcasting'] = fcst[0][0]

In [ ]:
for period in results.index:
    realtimeEndog = endog.loc[:period - 2].copy()
    mdl = VAR(realtimeEndog).fit()
    fcst = mdl.forecast(y=realtimeEndog.iloc[-mdl.k_ar:].to_numpy(), steps=2)
    results.loc[period, 'var_nowcasting_1Q_ahead'] = fcst[1][0]

In [ ]:
results

,var_nowcasting,var_nowcasting_1Q_ahead
2008Q1,1.654123,1.172038
2008Q2,1.522459,1.332697
2008Q3,0.916612,1.459869
2008Q4,1.121786,1.33451
2009Q1,-2.692955,1.343839
2009Q2,0.750234,-3.677332
2009Q3,2.187735,0.444185
2009Q4,1.34277,1.099914


In [ ]:
results.to_excel('var_results.xlsx')